# Soccer Forecasting Project

## Setup

### Libraries

In [1]:
import pandas as pd

### Load Data

In [30]:
# Modify this path to your local path
folder_path = '/Users/heshamnawaz/Desktop/Projects/soccer-forecasting-project'
elo_data = pd.read_csv(f"{folder_path}/data/EloRatings.csv")
matches = pd.read_csv(f"{folder_path}/data/matches.csv")

/var/folders/4f/h14zcr_15ss0fltppyfwj1q80000gn/T/ipykernel_60345/17581119.py:4: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  matches = pd.read_csv(f"{folder_path}/data/matches.csv")


In [8]:
elo_data

,date,club,country,elo
0,2000-07-01,Aachen,GER,1453.60
1,2000-07-01,Aalborg,DEN,1482.61
2,2000-07-01,Aalst,BEL,1337.53
3,2000-07-01,Aarhus,DEN,1381.46
4,2000-07-01,Aberdeen,SCO,1360.43
...,...,...,...,...
242586,2025-03-01,Wolfsburg,GER,1681.99
242587,2025-03-01,Wolves,ENG,1677.75
242588,2025-03-01,Zaragoza,ESP,1482.18
242589,2025-03-01,Zenit,RUS,1634.92


In [21]:
matches.FTResult

0         H
1         H
2         A
3         H
4         A
         ..
228372    D
228373    H
228374    D
228375    A
228376    H
Name: FTResult, Length: 228377, dtype: object

In [ ]:
Our rows are going to be the matches. For each match, we have the following information:
# - Date of the match
# - Home Team
# - Away Team
# - Home Team Goals
# - Away Team Goals
# - Home Team Elo Rating on the most recent date before the match
# - Away Team Elo Rating on the most recent date before the match``

### Matches

In [25]:
matches.columns

Index(['Division', 'MatchDate', 'MatchTime', 'HomeTeam', 'AwayTeam', 'HomeElo',
       'AwayElo', 'Form3Home', 'Form5Home', 'Form3Away', 'Form5Away', 'FTHome',
       'FTAway', 'FTResult', 'HTHome', 'HTAway', 'HTResult', 'HomeShots',
       'AwayShots', 'HomeTarget', 'AwayTarget', 'HomeFouls', 'AwayFouls',
       'HomeCorners', 'AwayCorners', 'HomeYellow', 'AwayYellow', 'HomeRed',
       'AwayRed', 'OddHome', 'OddDraw', 'OddAway', 'MaxHome', 'MaxDraw',
       'MaxAway', 'Over25', 'Under25', 'MaxOver25', 'MaxUnder25', 'HandiSize',
       'HandiHome', 'HandiAway'],
      dtype='object')

In [33]:
matches = \
(matches.assign(
    date = pd.to_datetime(matches.MatchDate),
    day = pd.to_datetime(matches.MatchDate).dt.day,
    month = pd.to_datetime(matches.MatchDate).dt.month,
    year = pd.to_datetime(matches.MatchDate).dt.year
).sort_values(by='date', ascending=True)
)

matches_rel_cols = ['Division', 'date', 'day', 'month', 'year', 'HomeElo', 'AwayElo', 'AwayTeam', 'FTResult']
matches_col_filtered = matches[matches_rel_cols]
matches_col_filtered

,Division,date,day,month,year,HomeElo,AwayElo,AwayTeam,FTResult
0,F1,2000-07-28,28,7,2000,1686.34,1586.57,Troyes,H
1,F1,2000-07-28,28,7,2000,1714.89,1642.51,Strasbourg,H
2,F2,2000-07-28,28,7,2000,1465.08,1633.80,Nancy,A
18,F1,2000-07-29,29,7,2000,1551.65,1656.37,Bastia,A
17,F1,2000-07-29,29,7,2000,1664.36,1730.89,Lens,A
...,...,...,...,...,...,...,...,...,...
228373,E0,2025-02-26,26,2,2025,1757.62,1584.51,Ipswich,H
228374,E0,2025-02-26,26,2,2025,1788.28,1999.49,Arsenal,D
228375,E0,2025-02-26,26,2,2025,1785.53,1926.48,Man City,A
228372,E0,2025-02-26,26,2,2025,1738.05,1731.52,Everton,D


In [41]:
filter_start_date = '2023-08-01'
filter_end_date = '2024-07-31'
divisions_list = ['E0']

# Apply both filters using logical AND (&)
filtered_matches = matches_col_filtered[
    (matches_col_filtered.date >= filter_start_date) & (matches_col_filtered.date <= filter_end_date) & 
    (matches_col_filtered.Division.isin(divisions_list))
]

filtered_matches

,Division,date,day,month,year,HomeElo,AwayElo,AwayTeam,FTResult
210362,E0,2023-08-11,11,8,2023,1726.42,2077.27,Man City,A
210454,E0,2023-08-12,12,8,2023,1876.02,1828.01,Aston Villa,H
210372,E0,2023-08-12,12,8,2023,1919.38,1673.98,Nottm Forest,H
210375,E0,2023-08-12,12,8,2023,1660.66,1779.48,West Ham,D
210376,E0,2023-08-12,12,8,2023,1828.20,1606.54,Luton,H
...,...,...,...,...,...,...,...,...,...
221142,E0,2024-05-19,19,5,2024,1732.68,1759.74,Man United,A
221141,E0,2024-05-19,19,5,2024,1721.75,1798.56,Newcastle,A
221140,E0,2024-05-19,19,5,2024,1944.71,1709.04,Everton,H
221143,E0,2024-05-19,19,5,2024,1620.34,1638.63,Nottm Forest,A
